# 🎵 Spodify → MP3

This notebook converts a Spotify playlist export into MP3 files using YouTube as an audio source.

It is designed to run in **Google Colab** and requires no local setup.
You will upload a playlist CSV, and the notebook will handle the rest.

> **Note:** This notebook is intended for personal use, experimentation, and data portability.



## 📦 Step 1 — Load Libraries

This cell installs and imports all required libraries.

It uses `yt-dlp` to search YouTube and extract audio.


In [ ]:
!pip install -U yt-dlp

## 📤 Step 2 — Load Your Spotify Playlist

Upload your Spotify playlist CSV file to the notebook files panel.

The CSV should include at least:
- `artist`
- `track`

This cell loads the playlist into a pandas DataFrame for processing.


(Make Spodify Playlist into CVS file using <https://exportify.net/>)

In [ ]:
import pandas as pd

CSV_PATH = "Preach.csv"  # put file in same folder as notebook/ or keep and download onepiecepl.csv to test

df = pd.read_csv(CSV_PATH)

# Keep only what we need
df["artist"] = df["Artist Name(s)"].str.split(",").str[0]
df["song"] = df["Track Name"]

df[["artist", "song"]].head()


## 🔗 Step 3 — Build Search Queries

This step converts each artist–track pair into a search query.

These queries will be used to find the best matching audio on YouTube.



In [ ]:
import subprocess

def yt_search(query):
    cmd = [
        "yt-dlp",
        f"ytsearch1:{query}",
        "--get-id",
        "--quiet"
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    video_id = result.stdout.strip()
    return f"https://www.youtube.com/watch?v={video_id}" if video_id else None


## 🔍 Step 4 — Search YouTube

For each track, this cell searches YouTube and selects the best available match.

The resulting video URLs are stored for download.




In [ ]:
from tqdm.notebook import tqdm
import sys
import subprocess

def yt_search(query):
    cmd = [
        sys.executable, "-m", "yt_dlp",
        f"ytsearch1:{query}",
        "--get-id",
        "--quiet"
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    video_id = result.stdout.strip()
    return f"https://www.youtube.com/watch?v={video_id}" if video_id else None


youtube_links = []

for _, row in tqdm(
    df.iterrows(),
    total=len(df),
    desc="Searching YouTube",
    unit="song"
):
    query = f"{row['artist']} - {row['song']}"
    youtube_links.append(yt_search(query))

df["youtube_link"] = youtube_links




## 🎧 Step 5 — Download MP3 Files

This final step downloads each track as an MP3 file.

Files are saved using the format:
`Artist - Song Title.mp3`

A progress indicator shows download status.


In [ ]:
import os
import re
from tqdm.notebook import tqdm

# Create music folder in Colab working directory (/content/music)
MUSIC_DIR = "/content/music"
os.makedirs(MUSIC_DIR, exist_ok=True)

def safe_filename(s):
    return re.sub(r'[<>:"/\\|?*]', '', s)

for _, row in tqdm(
    df.iterrows(),
    total=len(df),
    desc="Downloading MP3s",
    unit="song"
):
    if pd.isna(row["youtube_link"]):
        continue

    artist = safe_filename(row["artist"])
    song = safe_filename(row["song"])

    output_template = f"{MUSIC_DIR}/{artist} - {song}.%(ext)s"

    !python -m yt_dlp \
        -x --audio-format mp3 \
        -o "{output_template}" \
        "{row['youtube_link']}"

